In [37]:

import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from skimage.transform import resize
from itertools import product #중복순열
import random
############                    result data                   ###############
    #gaussian_denoise_data_5x5 = 가우시안 알파 트림드 5x5
    #snp_denoise_data_5x5 = salt&pepper 알파 트림드 5x5
    #gaussian_denoise_data_3x3 = 가우시안 알파 트림드 3x3
    #snp_denoise_data_3x3 = salt&pepper 알파 트림드 3x3
    #auto_predicted_gaussian_noised_test_data_32 = 32채널 가우시안 제거
    #auto_predicted_gaussian_noised_test_data_64 = 64채널 가우시안 제거
    #auto_predicted_snp_noised_test_data_32 = 32채널 salt&pepper 제거
    #auto_predicted_snp_noised_test_data_64 = 64채널 salt&pepper 제거
    #auto_test_data  = normalized test image
    #test_data = test image
##############################################################################

if __name__ == '__main__' :

    #################                 load data             ##################
    dataset, info = tfds.load('lfw', split='train[:3100]', with_info=True)#30->3000
    dataset_list = [img['image'] for img in tfds.as_numpy(dataset)]
    dataset_color = np.array(dataset_list)
    dataset_gray = np.dot(dataset_color[..., :3], [0.299, 0.587, 0.114]).astype(np.uint8)
    dataset_resize = (resize(dataset_gray, (3100, 256, 256))*255).astype(np.uint8)#30->3000
    train_data = dataset_resize[:3000]
    test_data = dataset_resize[3000:]
    data_shape = train_data[0].shape

    print(train_data.shape)
    print(test_data.shape)


############################### noised_data ####################################
    gaussian_noised_train_data = train_data.copy()  #gaussian_noised_data      #
                                                                               #
    snp_noised_train_data = train_data.copy()                                  #
                                                                               #
    gaussian_noised_test_data = test_data.copy()                               #
                                                                               #
    snp_noised_test_data = test_data.copy()                                    #
################################################################################

    # 이미지 크기 설정
    image_size = (256, 256)

    # 평균과 표준 편차 설정
    mean = 0
    std_dev = 10

    # 가우시안 노이즈 생성
    noise = np.random.normal(mean, std_dev, image_size)

####################### Add gaussian noise ########################################
                  ################# train data #####################              #
    for i in range(3000) :                                                        #
      for j in range(256) :                                                       #
        for k in range(256) :
          value = train_data[i][j][k] + noise[j][k]
          if value >= 255 :
            gaussian_noised_train_data[i][j][k] = 255                             #
          elif value <= 0 :
            gaussian_noised_train_data[i][j][k] = 0
          else :
            gaussian_noised_train_data[i][j][k] = value

                                                                                  #
              ##################### test data ######################              #
    for i in range(100) :                                                         #
      for j in range(256) :                                                       #
        for k in range(256) :
          value = test_data[i][j][k] + noise[j][k]
          if value >= 255 :
            gaussian_noised_test_data[i][j][k] = 255                              #
          elif value <= 0 :
            gaussian_noised_test_data[i][j][k] = 0
          else :
            gaussian_noised_test_data[i][j][k] = value
###################################################################################



    #################         get random position          #####################
    all_cases = []                                                             #
    snp_noise_random_pixel = []                                                #
    for i in product(range(256), range(256)) :                                 #
      all_cases.append(i)                                                      #
    snp_noise_random_pixel = random.sample(all_cases, 13107)                   #
################################################################################





###################### Add Salt and pepper noise ###############################
            ##############  train_data ####################
    for i in range(3000) :                                                     #
      for pixel in snp_noise_random_pixel :                                    #
          x = pixel[0]                                                         #
          y = pixel[1]                                                         #
          num = random.randrange(0,2)                                          #
          if num == 0 :                                                        #
            snp_noised_train_data[i][x][y] = 0                                 #
          else :                                                               #
            snp_noised_train_data[i][x][y] = 255                               #
            ############### test_data ##################
    for i in range(100) :                                                      #
      for pixel in snp_noise_random_pixel :                                    #
        x = pixel[0]                                                           #
        y = pixel[1]                                                           #
        num = random.randrange(0, 2)                                           #
        if num == 0 :                                                          #
          snp_noised_test_data[i][x][y] = 0                                    #
        else :                                                                 #
          snp_noised_test_data[i][x][y] = 255                                  #
################################################################################

    ############################# alpha trimmed 3x3 ############################
    gaussian_denoise_data_3x3 = gaussian_noised_test_data.copy()
    snp_denoise_data_3x3 = snp_noised_test_data.copy()
    for i in range(100) :
      for j in range(256) :
        for k in range(256) :
          avg_g = 0
          avg_s = 0
          gaussian_elements = []
          snp_elements = []
          for xpos in range(j-1, j + 2) :
            if xpos < 0 :                         # zero padding => append 0 to array
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)

              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
            elif xpos > 255 :                    # zero padding => append 0 to array
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)

              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
            else :
              for ypos in range(k-1, k+2) :
                if ypos == -1 or ypos == 256 :   # zero padding => append 0 to array
                  snp_elements.append(0)
                  snp_elements.append(0)
                  snp_elements.append(0)

                  gaussian_elements.append(0)
                  gaussian_elements.append(0)
                  gaussian_elements.append(0)
                else :                        # append pixel value to array
                  gaussian_elements.append(gaussian_noised_test_data[i][xpos][ypos])
                  snp_elements.append(snp_noised_test_data[i][xpos][ypos])
          gaussian_elements.sort()            # sort array
          snp_elements.sort()                 # sort array


          ################# get the average value #############################
          for e in range(1, 8) :
            avg_g += gaussian_elements[e]
            avg_s += snp_elements[e]
          avg_g /= 7
          avg_s /= 7
          ####################################################################



          ############################### set pixel value ####################
          gaussian_denoise_data_3x3[i][j][k] = avg_g
          snp_denoise_data_3x3[i][j][k] = avg_s
          ####################################################################


      #############################################################################

    ############################# alpha trimmed 5x5 ############################
    gaussian_denoise_data_5x5 = gaussian_noised_test_data.copy()
    snp_denoise_data_5x5 = snp_noised_test_data.copy()
    for i in range(100) :
      for j in range(256) :
        for k in range(256) :
          avg_g = 0
          avg_s = 0
          gaussian_elements = []
          snp_elements = []
          for xpos in range(j-2, j + 3) :
            if xpos < 0 :                       # zero padding => append 0 to array
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)

              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
            elif xpos > 255 :                   # zero padding => append 0 to array
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)
              gaussian_elements.append(0)

              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
              snp_elements.append(0)
            else :
              for ypos in range(k-2, k+3) :
                if ypos <= -1 or ypos >= 256 :  # zero padding => append 0 to array
                  gaussian_elements.append(0)
                  gaussian_elements.append(0)
                  gaussian_elements.append(0)
                  gaussian_elements.append(0)
                  gaussian_elements.append(0)

                  snp_elements.append(0)
                  snp_elements.append(0)
                  snp_elements.append(0)
                  snp_elements.append(0)
                  snp_elements.append(0)
                else :                           # append pixel value to array
                  gaussian_elements.append(gaussian_noised_test_data[i][xpos][ypos])
                  snp_elements.append(snp_noised_test_data[i][xpos][ypos])
          gaussian_elements.sort()               # sort array
          snp_elements.sort()

          ################# get the average value #############################
          for e in range(10, 15) :
            avg_g += gaussian_elements[e]
            avg_s += snp_elements[e]
          avg_g /= 5
          avg_s /= 5
          #####################################################################


          ############################### set pixel value ####################
          gaussian_denoise_data_5x5[i][j][k] = avg_g
          snp_denoise_data_5x5[i][j][k] = avg_s
          ####################################################################


        #############################################################################



    auto_train_data = (train_data.astype(np.float32)/255.).reshape(3000,256,256,1)
    auto_gaussian_noised_train_data_32 = (gaussian_noised_train_data.astype(np.float32)/255.).reshape(3000, 256, 256, 1)
    auto_test_data = (test_data.astype(np.float32) / 255.).reshape(100, 256,256,1)
    auto_gaussian_noised_test_data_32 = (gaussian_noised_test_data.astype(np.float32) / 255.).reshape(100, 256, 256, 1)


    #######################################       make 32 channel sequential model    ###################################
    model = keras.Sequential([
      keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
      keras.layers.MaxPooling2D((2, 2), padding = "same"),
      keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
      keras.layers.MaxPooling2D((2, 2), padding="same"),

      keras.layers.Conv2DTranspose(32, (3, 3), strides = 2, activation='relu', padding='same'),
      keras.layers.Conv2DTranspose(32, (3, 3), strides = 2, activation='relu', padding='same'),
      keras.layers.Conv2DTranspose(1, (3,3), activation='sigmoid', padding='same'),
    ])
    #####################################################################################################################



    #######################################       32 channel 가우시안 데이터    ###################################
    model.compile(optimizer='adam', loss='mse')                       #compile
    model.fit(auto_gaussian_noised_train_data_32, auto_train_data, epochs = 10)     #fit
    model.evaluate(auto_gaussian_noised_test_data_32, auto_test_data, verbose=2)


    auto_predicted_gaussian_noised_test_data_32 = model(auto_gaussian_noised_test_data_32)    # get test data
    ###############################################################################################################




    #######################################       32 channel salt and pepper 데이터    ###################################
    auto_snp_train_data = (train_data.astype(np.float32)/255.).reshape(3000,256,256,1)
    auto_snp_noised_train_data_32 = (snp_noised_train_data.astype(np.float32)/255.).reshape(3000, 256, 256, 1)
    auto_snp_test_data = (test_data.astype(np.float32) / 255.).reshape(100, 256,256,1)
    auto_snp_noised_test_data_32 = (snp_noised_test_data.astype(np.float32) / 255.).reshape(100, 256, 256, 1)

    model.compile(optimizer='adam', loss='mse')                               #compile
    model.fit(auto_snp_noised_train_data_32, auto_snp_train_data, epochs = 10)  #fit
    model.evaluate(auto_snp_noised_test_data_32, auto_snp_test_data, verbose=2)

    auto_predicted_snp_noised_test_data_32 = model(auto_snp_noised_test_data_32)    #get test data
    #####################################################################################################################



    auto_train_data = (train_data.astype(np.float32)/255.).reshape(3000,256,256,1)
    auto_gaussian_noised_train_data_64 = (gaussian_noised_train_data.astype(np.float32)/255.).reshape(3000, 256, 256, 1)
    auto_test_data = (test_data.astype(np.float32) / 255.).reshape(100, 256,256,1)
    auto_gaussian_noised_test_data_64 = (gaussian_noised_test_data.astype(np.float32) / 255.).reshape(100, 256, 256, 1)




    #######################################       make 64 channel sequential model    ###################################
    model = keras.Sequential([
      keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
      keras.layers.MaxPooling2D((2, 2), padding = "same"),
      keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
      keras.layers.MaxPooling2D((2, 2), padding="same"),

      keras.layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same'),
      keras.layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same'),
      keras.layers.Conv2DTranspose(1, (3,3), activation='sigmoid', padding='same'),
    ])
    ######################################################################################################################



    #######################################       64 channel 가우시안 데이터    ###################################
    model.compile(optimizer='adam', loss='mse')
    model.fit(auto_gaussian_noised_train_data_64, auto_train_data, epochs = 10)
    model.evaluate(auto_gaussian_noised_test_data_64, auto_test_data, verbose=2)
    auto_predicted_gaussian_noised_test_data_64 = model(auto_gaussian_noised_test_data_64)
    ################################################################################################################


    #######################################       64 channel salt and pepper 데이터    ###################################
    auto_snp_train_data = (train_data.astype(np.float32)/255.).reshape(3000,256,256,1)
    auto_snp_noised_train_data_64 = (snp_noised_train_data.astype(np.float32)/255.).reshape(3000, 256, 256, 1)
    auto_snp_test_data = (test_data.astype(np.float32) / 255.).reshape(100, 256,256,1)
    auto_snp_noised_test_data_64 = (snp_noised_test_data.astype(np.float32) / 255.).reshape(100, 256, 256, 1)

    model.compile(optimizer='adam', loss='mse')
    model.fit(auto_snp_noised_train_data_64, auto_snp_train_data, epochs = 10)
    model.evaluate(auto_snp_noised_test_data_64, auto_snp_test_data, verbose=2)
    auto_predicted_snp_noised_test_data_64 = model(auto_snp_noised_test_data_64)
    ###############################################################################################################


    mse_gaussian_denoise_data_5x5 = 0
    mse_gaussian_denoise_data_3x3 = 0
    mse_snp_denoise_data_5x5 = 0
    mse_snp_denoise_data_3x3 = 0
    mse_auto_predicted_gaussian_noised_test_data_32 = 0
    mse_auto_predicted_gaussian_noised_test_data_64 = 0
    mse_auto_predicted_snp_noised_test_data_32 = 0
    mse_auto_predicted_snp_noised_test_data_64 = 0


    ##########################################      calculate mse     #########################################
    for i in range(100) :
      mse_gaussian_denoise_data_5x5 += np.mean((test_data[i]- gaussian_denoise_data_5x5[i])**2)

      mse_gaussian_denoise_data_3x3 += np.mean((test_data[i]- gaussian_denoise_data_3x3[i])**2)

      mse_snp_denoise_data_5x5 += np.mean((test_data[i]- snp_denoise_data_5x5[i])**2)

      mse_snp_denoise_data_3x3 += np.mean((test_data[i]- snp_denoise_data_3x3[i])**2)

      mse_auto_predicted_gaussian_noised_test_data_32 += np.mean((auto_test_data[i]-auto_predicted_gaussian_noised_test_data_32[i])**2)

      mse_auto_predicted_gaussian_noised_test_data_64 += np.mean((auto_test_data[i]-auto_predicted_gaussian_noised_test_data_64[i])**2)

      mse_auto_predicted_snp_noised_test_data_32 += np.mean((auto_test_data[i]-auto_predicted_snp_noised_test_data_32[i])**2)

      mse_auto_predicted_snp_noised_test_data_64 += np.mean((auto_test_data[i]-auto_predicted_snp_noised_test_data_64[i])**2)
    ###############################################################################################################




    ###################                             calculate psnr          ##########################################################
    psnr_gaussian_denoise_data_5x5 = 10*np.log10(255*255/(mse_gaussian_denoise_data_5x5/100))
    psnr_gaussian_denoise_data_3x3 = 10*np.log10(255*255/(mse_gaussian_denoise_data_3x3/100))
    psnr_snp_denoise_data_5x5 = 10*np.log10(255*255/(mse_snp_denoise_data_5x5/100))
    psnr_snp_denoise_data_3x3 = 10*np.log10(255*255/(mse_snp_denoise_data_3x3/100))
    psnr_auto_predicted_gaussian_noised_test_data_32 = 10*np.log10(1/(mse_auto_predicted_gaussian_noised_test_data_32/100))
    psnr_auto_predicted_gaussian_noised_test_data_64 = 10*np.log10(1/(mse_auto_predicted_gaussian_noised_test_data_64/100))
    psnr_auto_predicted_snp_noised_test_data_32 = 10*np.log10(1/(mse_auto_predicted_snp_noised_test_data_32/100))
    psnr_auto_predicted_snp_noised_test_data_64 = 10*np.log10(1/(mse_auto_predicted_snp_noised_test_data_64/100))
    ################################################################################################################################




    print("psnr_gaussian_denoise_data_5x5 : " + str(psnr_gaussian_denoise_data_5x5))
    print("psnr_gaussian_denoise_data_3x3 : " + str(psnr_gaussian_denoise_data_3x3))
    print("psnr_snp_denoise_data_5x5 : " + str(psnr_snp_denoise_data_5x5))
    print("psnr_snp_denoise_data_3x3 : " + str(psnr_snp_denoise_data_3x3))
    print("psnr_auto_predicted_gaussian_noised_test_data_32 : " + str(psnr_auto_predicted_gaussian_noised_test_data_32))
    print("psnr_auto_predicted_gaussian_noised_test_data_64 : " + str(psnr_auto_predicted_gaussian_noised_test_data_64))
    print("psnr_auto_predicted_snp_noised_test_data_32 : " + str(psnr_auto_predicted_snp_noised_test_data_32))
    print("psnr_auto_predicted_snp_noised_test_data_64 : " + str(psnr_auto_predicted_snp_noised_test_data_64))



(3000, 256, 256)
(100, 256, 256)
Epoch 1/10
94/94 [==============================] - 8s 67ms/step - loss: 0.0275
Epoch 2/10
94/94 [==============================] - 6s 67ms/step - loss: 0.0020
Epoch 3/10
94/94 [==============================] - 6s 67ms/step - loss: 0.0013
Epoch 4/10
94/94 [==============================] - 6s 67ms/step - loss: 0.0011
Epoch 5/10
94/94 [==============================] - 6s 67ms/step - loss: 9.0031e-04
Epoch 6/10
94/94 [==============================] - 6s 67ms/step - loss: 8.1035e-04
Epoch 7/10
94/94 [==============================] - 6s 67ms/step - loss: 7.1385e-04
Epoch 8/10
94/94 [==============================] - 6s 67ms/step - loss: 7.6728e-04
Epoch 9/10
94/94 [==============================] - 6s 66ms/step - loss: 6.0954e-04
Epoch 10/10
94/94 [==============================] - 6s 67ms/step - loss: 5.6484e-04
4/4 - 0s - loss: 5.7807e-04 - 226ms/epoch - 57ms/step
Epoch 1/10
94/94 [==============================] - 8s 67ms/step - loss: 0.0066
Epoch 2/